In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
stop = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import os
os.chdir('/content/drive/My Drive')

In [2]:
import pandas as pd
import re
import numpy as np
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

from sklearn.multiclass import OneVsRestClassifier


In [10]:
data= pd.read_csv('Train_classification.csv',encoding='latin-1')

In [11]:
def preprocess(text):
    text=text.lower()
    text=re.sub('[^0-9a-zA-Z]+',' ',text)
    words = text.split()
    words2 = [word for word in words if word not in stop]
    words3=' '.join(words2)
    return(words3)

In [12]:
def text_cleaner(text):
    text = re.sub("\'", "", str(text))     
    text = re.sub("[^a-zA-Z]"," ",str(text))
    text = ' '.join( [w for w in text.split() if len(w)>1] )
    text = ' '.join(text.split()) 
    text = text.lower()
    return text

In [13]:
data['Text']= data['Text'].apply(lambda x: preprocess(x))
data['Tags']= data['Tags'].apply(lambda x: preprocess(x))

In [14]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

data['Tags']= data['Tags'].apply(lambda x: word_tokenize(x))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
data

,Text,Tags
0,standard materi,"[estimation, painting, proposal]"
1,demineralized water partially deionized could ...,[proposal]


In [16]:
multilabel = MultiLabelBinarizer()

In [17]:
y = multilabel.fit_transform(data['Tags'])

In [18]:
y

array([[1, 1, 1],
       [0, 0, 1]])

In [19]:
multilabel.classes_

array(['estimation', 'painting', 'proposal'], dtype=object)

In [20]:
y.shape

(2, 3)

In [21]:
tfidf = TfidfVectorizer(analyzer='word', max_features=10000, ngram_range=(1,2), stop_words='english')
X = tfidf.fit_transform(data['Text'])

In [22]:
X.shape

(2, 16)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [24]:
sgd = SGDClassifier()
lr = LogisticRegression(solver='lbfgs')
svc = LinearSVC()

In [25]:
def j_score(y_true, y_pred):
    jaccard = np.minimum(y_true, y_pred).sum(axis = 1)/np.maximum(y_true, y_pred).sum(axis = 1)
    return jaccard.mean()*100


def print_score(y_pred, clf):
    print("Clf: ", clf.__class__.__name__)
    print('Jacard score: {}'.format(j_score(y_test, y_pred)))
    print('----')

In [26]:
for classifier in [LinearSVC(C=1.5, penalty = 'l1', dual=False)]:
    clf = OneVsRestClassifier(classifier)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print_score(y_pred, classifier)

Clf:  LinearSVC
Jacard score: 33.33333333333333
----


C:\Users\HP\anaconda3\lib\site-packages\sklearn\multiclass.py:77: UserWarning: Label 0 is present in all training examples.
  str(classes[c]))
C:\Users\HP\anaconda3\lib\site-packages\sklearn\multiclass.py:77: UserWarning: Label 1 is present in all training examples.
  str(classes[c]))
C:\Users\HP\anaconda3\lib\site-packages\sklearn\multiclass.py:77: UserWarning: Label 2 is present in all training examples.
  str(classes[c]))


In [35]:
x =  "STANDARD The quantity of M.I.O. in the main pigment must be 80% by"
x=[preprocess(x)]

In [36]:
xt = tfidf.transform(x)

In [37]:
r=clf.predict(xt)

In [38]:
r=multilabel.inverse_transform(r)

In [56]:
pickle.dump(clf,open("Multiclf.pickle",'wb'))
pickle.dump(tfidf,open("Multitfidf.pickle",'wb'))
pickle.dump(multilabel,open("label.pickle",'wb'))

In [55]:
import pickle

In [39]:
r

[('estimation', 'painting', 'proposal')]

In [163]:
s=[','.join(i) for i in r]

In [125]:
type(s)

TypeError: 'list' object is not callable

In [122]:
s=str(s)

In [128]:
s

['estimation,proposal']

In [259]:
res = list(map(','.join, r))

In [260]:
res = str(res)[1:-1].capitalize()

In [267]:
(res.split(","))

['estimation', 'proposal']

In [263]:
res=(res)[1:-1] 

In [262]:
res

"'estimation,proposal'"

In [156]:
res=list(map(','.join, res))
#res = (res)[1:-1] 

In [157]:
res

['estimation,proposal']